In [11]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline  
%load_ext Cython

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
import optuna


from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
# data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
data_train_path="data_train.csv"
data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [8]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

<13024x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [9]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2514 (19.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2625 (20.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2573 (19.8%) Users that have less than 1 test interactions


In [19]:
item_knn_recommenders = ItemKNNCFRecommender(URM_trains[0], verbose=False), ItemKNNCFRecommender(URM_trains[1], verbose=False), ItemKNNCFRecommender(URM_trains[2], verbose=False)
RP3beta_recommenders = RP3betaRecommender(URM_trains[0], verbose=False), RP3betaRecommender(URM_trains[1], verbose=False), RP3betaRecommender(URM_trains[2], verbose=False)
P3alpha_recommenders = P3alphaRecommender(URM_trains[0], verbose=False), P3alphaRecommender(URM_trains[1], verbose=False), P3alphaRecommender(URM_trains[2], verbose=False)
# user_knn_recommender = UserKNNCFRecommender(URM_train)

for i in range(0, test_folds):
    item_knn_recommenders[i].fit(shrink=6.982686167013746, topK=9)
    RP3beta_recommenders[i].fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
    P3alpha_recommenders[i].fit(topK=41, alpha=0.7384557977476195)
    # user_knn_recommender[i].fit(shrink=0, topK=483)



Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2321.21 column/sec. Elapsed time 9.57 sec
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2521.03 column/sec. Elapsed time 8.81 sec
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2626.21 column/sec. Elapsed time 8.46 sec


In [23]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3, verbose = True):

        super(ScoresHybridRecommender, self).__init__(URM_train, verbose=verbose)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
    def fit(self, alpha = 0.5, beta = 0.5):
        self.alpha = alpha    
        self.beta = beta


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*self.beta + item_weights_3*(1-self.alpha-self.beta)

        return item_weights

In [24]:
def train_evaluate(optuna_trial):
    alpha = optuna_trial.suggest_float("alpha", 0.0, 1.0)
    beta = optuna_trial.suggest_float("beta", 0.0, 1.0 - alpha)

    mAP = 0.0
    for i in range(0, test_folds):
        P3alpha = ScoresHybridRecommender(URM_trains[i], item_knn_recommenders[i], RP3beta_recommenders[i], P3alpha_recommenders[i], verbose=False)
        P3alpha.fit(alpha=alpha, beta=beta)
        result_df, _ = evaluator_tests[i].evaluateRecommender(P3alpha)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [25]:
study = optuna.create_study(direction="maximize")
study.optimize(train_evaluate, n_trials=100)

[I 2023-12-11 14:04:47,807] A new study created in memory with name: no-name-dd5651ca-4550-438f-952d-4a1e0f32ffba


EvaluatorHoldout: Processed 10510 (100.0%) in 10.22 sec. Users per second: 1029
EvaluatorHoldout: Processed 10399 (100.0%) in 10.00 sec. Users per second: 1040
EvaluatorHoldout: Processed 10451 (100.0%) in 10.16 sec. Users per second: 1029


[I 2023-12-11 14:05:18,243] Trial 0 finished with value: 0.04778811296852026 and parameters: {'alpha': 0.3696220146091105, 'beta': 0.36926436280949976}. Best is trial 0 with value: 0.04778811296852026.


EvaluatorHoldout: Processed 10510 (100.0%) in 10.33 sec. Users per second: 1017
EvaluatorHoldout: Processed 10399 (100.0%) in 10.06 sec. Users per second: 1034
EvaluatorHoldout: Processed 10451 (100.0%) in 10.08 sec. Users per second: 1037


[I 2023-12-11 14:05:48,773] Trial 1 finished with value: 0.04373641485255658 and parameters: {'alpha': 0.0049265111642734505, 'beta': 0.04470057217707943}. Best is trial 0 with value: 0.04778811296852026.


...

[I 2023-12-11 14:52:59,964] Trial 97 finished with value: 0.049729110953277735 and parameters: {'alpha': 0.13724001594711052, 'beta': 0.6882223639634905}. Best is trial 92 with value: 0.05003942306359089.


EvaluatorHoldout: Processed 10510 (100.0%) in 10.11 sec. Users per second: 1039
EvaluatorHoldout: Processed 10399 (100.0%) in 11.72 sec. Users per second: 887
EvaluatorHoldout: Processed 10451 (100.0%) in 10.64 sec. Users per second: 982


[I 2023-12-11 14:53:32,514] Trial 98 finished with value: 0.04988597873827463 and parameters: {'alpha': 0.09513265726866704, 'beta': 0.6610107026233978}. Best is trial 92 with value: 0.05003942306359089.


EvaluatorHoldout: Processed 10510 (100.0%) in 10.19 sec. Users per second: 1032
EvaluatorHoldout: Processed 10399 (100.0%) in 9.85 sec. Users per second: 1056
EvaluatorHoldout: Processed 10451 (100.0%) in 10.41 sec. Users per second: 1004


[I 2023-12-11 14:54:03,022] Trial 99 finished with value: 0.049990818789635204 and parameters: {'alpha': 0.073665521513732, 'beta': 0.7072140480122339}. Best is trial 92 with value: 0.05003942306359089.


In [26]:
import time
alpha = study.best_params["alpha"]
beta = study.best_params["beta"]
print(f"Weights: itemKNN= {alpha}, RP3beta= {beta}, P3alpha= {1-alpha-beta}")

item_knn_recommender = ItemKNNCFRecommender(URM_all, verbose=False)
RP3beta_recommender = RP3betaRecommender(URM_all, verbose=False)
P3alpha_recommender = P3alphaRecommender(URM_all, verbose=False)

item_knn_recommender.fit(shrink=6.982686167013746, topK=9)
RP3beta_recommender.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
P3alpha_recommender.fit(topK=41, alpha=0.7384557977476195)

scoreshybridrecommender = ScoresHybridRecommender(URM_all, item_knn_recommender, RP3beta_recommender, P3alpha_recommender, verbose=False)
scoreshybridrecommender.fit(alpha = alpha, beta = beta)
n_users_to_test = len(data_target["user_id"])
suggestions = pd.DataFrame(columns = ["user_id", "item_list"])
start_time = time.time()

for user_id in (data_target["user_id"]-1):
    suggestion = scoreshybridrecommender.recommend(user_id, cutoff=10)
    suggestions.loc[len(suggestions)] = [user_id+1, " ".join([str(item_map[x]) for x in suggestion])] #" ".join([str(x) for x in np.flip(np.sort(suggestion_values))[0][:10]]
    
end_time = time.time()

print("Reasonable implementation speed is {:.2f} usr/sec".format(n_users_to_test/(end_time-start_time)))
suggestions

Weights: itemKNN= 0.08488318219716774, RP3beta= 0.7337742901018609, P3alpha= 0.18134252770097137
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2506.82 column/sec. Elapsed time 8.86 sec
Reasonable implementation speed is 574.52 usr/sec


,user_id,item_list
0,1,36 101 403 506 123 515 694 1546 592 254
1,2,1095 47 28 11 50 1522 196 102 3176 12
2,3,59 259 857 584 536 956 4252 1281 648 414
3,4,28 50 249 145 639 136 139 171 254 314
4,5,1570 77 5138 116 148 7033 131 95 175 238
...,...,...
10877,13020,6450 6198 6452 6749 7395 7394 4323 161 105 1191
10878,13021,6179 7027 6720 6451 6426 133 13621 17942 7395 ...
10879,13022,1411 1446 1668 1674 809 4688 10789 16075 1555 ...
10880,13023,706 639 1124 329 1534 1532 1146 1273 837 1535


In [45]:
suggestions.to_csv("outputs/hybrid_itemKNN_RP3beta_P3alpha.csv", index=False)